In [1]:
from apiclient.discovery import build

def build_service(filename):
    with open(filename) as f:
        key = f.readline()

    YOUTUBE_API_SERVICE_NAME = "youtube"
    YOUTUBE_API_VERSION = "v3"
    return build(YOUTUBE_API_SERVICE_NAME,
                 YOUTUBE_API_VERSION,
                 developerKey=key)

In [44]:
#!pip install --upgrade google-api-python-client
apikey = ''
service = build_service('apikey.json') 

In [4]:
import numpy as np
import pandas as pd
from datetime import datetime

In [7]:
# import json
# import urllib.request
# import string
# import random

# count = 50

# random = ''.join(random.choice(string.ascii_uppercase + string.digits) for _ in range(3))
# print(random)
# random_videoIds = set()
# urlData = "https://www.googleapis.com/youtube/v3/search?key={}&maxResults={}&part=snippet&type=video&q={}".format(apikey,count,random)
# webURL = urllib.request.urlopen(urlData)
# data = webURL.read()
# encoding = webURL.info().get_content_charset('utf-8')
# results = json.loads(data.decode(encoding))
# print(len(random_videoIds))

# for data in results['items']:
#     videoId = (data['id']['videoId'])
#     random_videoIds.add(videoId)
    
# print(len(random_videoIds))

In [22]:
#get random youtube video_id
def get_random_video_id(i=1):
    import json
    import urllib.request
    import string
    import random

    count = 50
    # API_KEY = 'your_key'
#     random = ''.join(random.choice(string.ascii_uppercase + string.digits) for _ in range(3))
    random_videoIds = set()
    
    try:
        while(len(random_videoIds) <= 10000):
            random_key = ''.join(random.choice(string.ascii_uppercase + string.digits) for _ in range(3))
            urlData = "https://www.googleapis.com/youtube/v3/search?key={}&maxResults={}&part=snippet&type=video&q={}".format(apikey,count,random_key)
            webURL = urllib.request.urlopen(urlData)
            data = webURL.read()
            encoding = webURL.info().get_content_charset('utf-8')
            results = json.loads(data.decode(encoding))
            print(len(random_videoIds))

            for data in results['items']:
                videoId = (data['id']['videoId'])
                random_videoIds.add(videoId)
    except Exception as e:
        print(e)
        textfile = open("random_videos"+str(i)+".txt", "w")
        for element in random_videoIds:
            textfile.write(element + "\n")
        textfile.close()
        return random_videoIds

    textfile = open("random_videos"+str(i)+".txt", "w")
    for element in random_videoIds:
        textfile.write(element + "\n")
    textfile.close()
    return random_videoIds


In [1]:
random_videos = get_random_video_id(4)
print(len(random_videos))

In [28]:
def get_video_metadata(val=1):
    
    my_file = open("random_videos"+str(val)+".txt", "r")
    content = my_file.read()
    videoIdList = content.splitlines()
    my_file.close()

    #output list required
    video_id = [] #got
    title = [] #got
    publishedAt = [] #got
    channelId = [] #got
    channelTitle = [] #got
    categoryId = [] #got
    trending_date = [] #none
    tags = [] #got
    view_count = [] #got
    likes = [] #got
    dislikes = []
    comment_count = [] #got
    thumbnail_link = [] #none
    comments_disabled = [] #none
    ratings_disabled = [] #none
    description = [] #got


    for i in range(0, len(videoIdList)):
        try:
            response = service.videos().list(
                        part="statistics,snippet",
                        id=videoIdList[i]
                    ).execute()             
        except Exception as e:
            print(e)
            print("Failed on this video, trying next...", videoIdList[i])
            continue


        for item in response['items']:
    #         print(item)

            try:
                likes.append(item['statistics']['likeCount'])
            except:
                likes.append(None)

            try:
                video_id.append(item['id'])
            except:
                video_id.append(None)

            try:
                view_count.append(item['statistics']['viewCount'])
            except:
                view_count.append(None)

            try:
                publishedAt.append(item['snippet']['publishedAt'])
            except:
                publishedAt.append(None)

            try:
                channelId.append(item['snippet']['channelId'])
            except:
                channelId.append(None)

            try:
                categoryId.append(item['snippet']['categoryId'])
            except:
                categoryId.append(None)

            try:
                channelTitle.append(item['snippet']['channelTitle'])
            except:
                channelTitle.append(None)

            try:
                title.append(item['snippet']['title'])
            except:
                title.append(None)

            try:
                description.append(item['snippet']['description'])
            except:
                description.append(None)

            try:
                tags.append(item['snippet']['tags'] )
            except:
                tags.append(None)

            try:
                comment_count.append(item['statistics']['commentCount'] or None)
            except:
                comment_count.append(None)


    trending_date = [None] * len(likes)
    dislikes = [None] * len(likes)
    thumbnail_link= [None] * len(likes)
    comments_disabled = [None] * len(likes)
    ratings_disabled = [None] * len(likes)
    tags = [None] * len(likes)

#     print(len(likes))
#     print(len(dislikes))
#     print(len(thumbnail_link))
#     print(len(view_count))
#     print(len(video_id))
#     print(len(publishedAt))
#     print(len(channelId))
#     print(len(channelTitle))
#     print(len(categoryId))
#     print(len(title))
#     print(len(description))
#     print(len(tags))
#     print(len(trending_date))
#     print(len(comments_disabled))
#     print(len(ratings_disabled))


    non_trending_data = pd.DataFrame({'video_id': video_id,'title': title,'publishedAt': publishedAt,'channelId': channelId,
                                      'channelTitle': channelTitle,'categoryId' : categoryId, 'trending_date': trending_date,
                                      'tags': tags,'view_count': view_count,'likes': likes,'dislikes':dislikes,
                                      'comment_count': comment_count, 'thumbnail_link': thumbnail_link,
                                      'comments_disabled': comments_disabled,'ratings_disabled':ratings_disabled,
                                      'description':description
                                     })
    if not non_trending_data.empty:
        finaldf = non_trending_data.drop_duplicates().dropna(axis = 0, how = 'all', inplace = False)
        finaldf.shape
        finaldf.head()
        finaldf.to_csv(r"./NonTrendingData"+str(val)+".csv", index=False)

In [2]:
get_video_metadata(1)
get_video_metadata(2)
# get_video_metadata(3)
# get_video_metadata(4)

In [89]:
# non_trending_data = pd.DataFrame({'video_id': video_id,'title': title,'publishedAt': publishedAt,'channelId': channelId,
#                                       'channelTitle': channelTitle,'categoryId' : categoryId, 'trending_date': trending_date,
#                                       'tags': tags,'view_count': view_count,'likes': likes,'dislikes':dislikes,
#                                       'comment_count': comment_count, 'thumbnail_link': thumbnail_link,
#                                       'comments_disabled': comments_disabled,'ratings_disabled':ratings_disabled,
#                                       'description':description
#                                      })
# if not non_trending_data.empty:
#         finaldf = non_trending_data.drop_duplicates().dropna(axis = 0, how = 'all', inplace = False)
#         finaldf.shape
#         finaldf.head()
#         finaldf.to_csv(r"./NonTrendingData"+str(10)+".csv", index=False)

In [27]:
# my_file = open("random_videos9.txt", "r")
# content = my_file.read()
# videoIdList = content.splitlines()
# my_file.close()

# response = service.videos().list(
#                 part="statistics,snippet",
#                 id=videoIdList[0]
#             ).execute() 


In [ ]:
# for item in response['items']:
#     print(item)
    
# print(response)

In [47]:
# !pip install glob
import glob

csvFiles = glob.glob('FINALData*.csv')
print(csvFiles)
finaldf = pd.DataFrame(columns = ['video_id','title','publishedAt','channelId','channelTitle','categoryId', 'trending_date',
                                      'tags','view_count','likes','dislikes','comment_count', 'thumbnail_link',
                                      'comments_disabled','ratings_disabled','description'
                                 ])

#pandas read csv
#union csv
for file in csvFiles:
    tempdf = pd.read_csv(file)
    finaldf = pd.concat([finaldf, tempdf])

#drop duplicates
#write CSV
finaldf.drop_duplicates().to_csv(r"./ULTIMATEFINALData.csv", index=False)

['FINALData1.csv', 'FINALData2.csv']
